# CredNest AI - Model Fine-tuning

## System Configuration
- **CPU:** AMD Ryzen 7 5800H
- **GPU:** NVIDIA RTX 3060 (6GB VRAM)
- **RAM:** 16GB

## Optimization Strategy
- 4-bit quantization for memory efficiency
- LoRA for parameter-efficient fine-tuning
- Optimized for 6GB VRAM

In [1]:
# Install/Update required packages
# We use --force-reinstall to ensure we get the latest compatible versions
!pip install -q torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install "numpy<2.0"
!pip install --force-reinstall -U transformers
!pip install --force-reinstall -U tokenizers
!pip install -q -U peft
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q scipy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bitsandbytes 0.48.2 requires torch<3,>=2.3, but you have torch 2.1.0+cu118 which is incompatible.


  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached charset_nor

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bitsandbytes 0.48.2 requires torch<3,>=2.3, but you have torch 2.1.0+cu118 which is incompatible.
gtts 2.5.4 requires click<8.2,>=7.1, but you have click 8.3.1 which is incompatible.
tensorflow-intel 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.5.4 which is incompatible.
tensorflow-intel 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 2.2.6 which is incompatible.


  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached huggingface_hub-1.1.5-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl.metadata (5.0 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.metadata (2.4 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typer_slim-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.wh

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bitsandbytes 0.48.2 requires torch<3,>=2.3, but you have torch 2.1.0+cu118 which is incompatible.
gtts 2.5.4 requires click<8.2,>=7.1, but you have click 8.3.1 which is incompatible.
tensorflow-intel 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.5.4 which is incompatible.
tensorflow-intel 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 2.2.6 which is incompatible.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.1.5 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0+cu118, but you have torch 2.9.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1

### ⚠️ IMPORTANT: RESTART KERNEL
After running the installation cell above, you **MUST** restart the kernel for the changes to take effect.
Go to **Kernel** > **Restart** in the menu bar.

In [1]:
# Import libraries
import torch
import transformers
import tokenizers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from datasets import Dataset
from trl import SFTTrainer
import json
import os

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ Transformers version: {transformers.__version__}")
print(f"✅ Tokenizers version: {tokenizers.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Version check warning
if transformers.__version__ < "4.37.0":
    print("\n⚠️ WARNING: Transformers version is too old. Please RESTART THE KERNEL and run the installation cell again.")

c:\Users\anujn\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: C:\Users\anujn\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent c

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\anujn\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\anujn\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\anujn\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\anujn\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _ARRAY_API not found

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (c:\Users\anujn\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\__init__.py)

## Load Training Data

We have 27 comprehensive financial examples covering:
- Real bank loan schemes (SBI, HDFC, ICICI, Axis, PNB)
- EMI calculations
- Tax planning
- Investment strategies

In [ ]:
# Load training data
with open('server/complete_financial_training.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

print(f"✅ Loaded {len(training_data)} training examples")
print(f"\nSample example:")
print(f"Instruction: {training_data[0]['instruction']}")
print(f"Input: {training_data[0]['input'][:100]}...")

In [ ]:
# Format data for training
def format_instruction(sample):
    return f"""### Instruction:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
{sample['output']}"""

# Create dataset
dataset = Dataset.from_list(training_data)
dataset = dataset.map(lambda x: {"text": format_instruction(x)})

print(f"✅ Dataset prepared with {len(dataset)} examples")

## Load Model with 4-bit Quantization

Using Mistral-7B with 4-bit quantization to fit in 6GB VRAM

In [ ]:
# Model configuration
model_name = "mistralai/Mistral-7B-v0.1"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer
print("Loading tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    print("✅ Tokenizer loaded successfully!")
except Exception as e:
    print(f"❌ Error loading tokenizer: {e}")
    print("Try restarting the kernel and running the installation cell again.")

# Load model
print("Loading model with 4-bit quantization...")
print("This will take 2-3 minutes...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Prepare for k-bit training
model = prepare_model_for_kbit_training(model)
print("✅ Model loaded successfully!")
print(f"GPU Memory: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

## Configure LoRA

LoRA allows efficient fine-tuning by training only a small subset of parameters

In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
print("✅ LoRA configured!")

## Training Configuration

Optimized for RTX 3060 6GB VRAM

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./crednest-ai-model",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
)

print("✅ Trainer initialized!")

## Start Training

This will take approximately 10-15 minutes on RTX 3060

In [ ]:
# Clear CUDA cache
torch.cuda.empty_cache()

print("🚀 Starting training...")
print(f"GPU Memory before training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

# Train
trainer.train()

print("\n✅ Training completed!")
print(f"GPU Memory after training: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

## Save Fine-tuned Model

In [ ]:
# Save model
output_dir = "./crednest-ai-finetuned"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved to: {output_dir}")
print(f"\nModel files:")
for file in os.listdir(output_dir):
    size = os.path.getsize(os.path.join(output_dir, file)) / (1024**2)
    print(f"  - {file}: {size:.2f} MB")

## Test the Fine-tuned Model

In [ ]:
# Test function
def generate_response(instruction, input_text):
    prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Response:")[1].strip()

# Test cases
print("🧪 Test 1: EMI Calculation")
print("="*50)
response1 = generate_response(
    "Calculate EMI for a loan",
    "Calculate EMI for ₹5 lakh loan at 10% for 5 years"
)
print(response1)

print("\n" + "="*50 + "\n")

print("🧪 Test 2: Loan Eligibility")
print("="*50)
response2 = generate_response(
    "Check loan eligibility",
    "Am I eligible for ₹10 lakh loan with ₹50,000 monthly income?"
)
print(response2)

## Performance Statistics

In [ ]:
# GPU Memory Statistics
print("📊 GPU Memory Statistics:")
print(f"Allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved()/1024**3:.2f} GB")
print(f"Max Allocated: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")

# Model size
def get_folder_size(folder):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024**3)

print(f"\n📦 Model Size:")
print(f"Fine-tuned model: {get_folder_size(output_dir):.2f} GB")

print("\n✅ Fine-tuning complete!")
print("\n🎉 Your financial AI model is ready to use!")